In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as smf

e_transfers = pd.read_csv("transfery.csv")
e_transfers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   vek_mother          1012 non-null   object 
 1   vek_embryo          827 non-null    object 
 2   genetic_method      850 non-null    object 
 3   f_donor             186 non-null    float64
 4   examined_genes      86 non-null     object 
 5   sex                 835 non-null    object 
 6   clinical_gravidity  1000 non-null   float64
dtypes: float64(2), object(5)
memory usage: 55.5+ KB


In [2]:
# kontrola hodnot ve sloupcích vek_embryo
e_transfers["vek_embryo"].value_counts()
e_transfers["vek_embryo"].isnull().value_counts()

vek_embryo
False    827
True     185
Name: count, dtype: int64

In [3]:
# kontrola hodnot ve sloupcích f_donor
e_transfers["f_donor"].value_counts()

f_donor
1.0    186
Name: count, dtype: int64

In [4]:
# odstranění hodnot ve sloupci f_donor = 1.0
e_transfers.drop(e_transfers[e_transfers['f_donor'] == 1.0].index, inplace=True)
e_transfers["f_donor"].count()

0

In [5]:
# odstanění null hodnot a x ve sloupcích vek_embryo
e_transfers.drop(e_transfers[e_transfers['vek_embryo'] == "x"].index, inplace=True)
e_transfers.dropna(subset=['vek_embryo'], inplace=True)

In [6]:
e_transfers["vek_embryo"].isnull().value_counts()

vek_embryo
False    825
Name: count, dtype: int64

In [7]:
##rozdělení vek_embryo do 5 kategorií
vek_embryo_1 = e_transfers[e_transfers["vek_embryo"] <= "29"]
vek_embryo_2 = e_transfers[(e_transfers["vek_embryo"] >= "30") & (e_transfers["vek_embryo"] <= "34")]
vek_embryo_3 = e_transfers[(e_transfers["vek_embryo"] >= "35") & (e_transfers["vek_embryo"] <= "39")]
vek_embryo_4 = e_transfers[e_transfers["vek_embryo"] >= "40"]
vek_embryo_all = e_transfers["vek_embryo"]

print(vek_embryo_1)

     vek_mother vek_embryo genetic_method  f_donor examined_genes  sex  \
3            32         28   Karyomapping      NaN           TGM1   XX   
7            29         29            NaN      NaN            NaN  NaN   
8            26         24          PGT-A      NaN            NaN   XY   
9            28         27            NaN      NaN            NaN  NaN   
16           26         25          PGT-A      NaN            NaN   XX   
...         ...        ...            ...      ...            ...  ...   
972          31         29          PGT-A      NaN            NaN   XY   
993          28         27          PGT-A      NaN            NaN   XX   
996          28         27          PGT-A      NaN            NaN   XX   
1002         29         27          PGT-A      NaN            NaN   XY   
1003         29         28          PGT-A      NaN            NaN   XX   

      clinical_gravidity  
3                    1.0  
7                    0.0  
8                    1.0  
9  

In [8]:
# spočítání úspěšnosti transferu
e_success_all = ((e_transfers["clinical_gravidity"] == 1.0).sum() /(e_transfers["clinical_gravidity"].count()) * 100).round(2)
e_success_1 = ((vek_embryo_1["clinical_gravidity"] == 1.0).sum() /vek_embryo_1["clinical_gravidity"].count() * 100).round(2)
e_success_2 = ((vek_embryo_2["clinical_gravidity"] == 1.0).sum() /vek_embryo_2["clinical_gravidity"].count() * 100).round(2)
e_success_3 = ((vek_embryo_3["clinical_gravidity"] == 1.0).sum() /vek_embryo_3["clinical_gravidity"].count() * 100).round(2)
e_success_4 = ((vek_embryo_4["clinical_gravidity"] == 1.0).sum() /vek_embryo_4["clinical_gravidity"].count() * 100).round(2)

In [9]:
print(e_success_all)
print(e_success_1)
print(e_success_2)
print(e_success_3)
print(e_success_4)


51.34
54.59
51.54
47.89
50.0


In [10]:
# Zobrazení tabulky
data_embryo = {
    'Všechny věkové kategorie': [e_success_all],
    'do 29': [e_success_1],
    '30-34': [e_success_2],
    '35-39': [e_success_3],
    '40 a výše': [e_success_4]
}

new_dataframe_embryo = pd.DataFrame(data_embryo)
new_dataframe_embryo

,Všechny věkové kategorie,do 29,30-34,35-39,40 a výše
0,51.34,54.59,51.54,47.89,50.0


In [12]:
# Spearmanův korelační koeficient & Kendallovo Tau

spearman = stats.spearmanr(e_transfers["vek_embryo"], e_transfers["clinical_gravidity"])
kendall = stats.kendalltau(e_transfers["vek_embryo"], e_transfers["clinical_gravidity"])

print(f"Spearman: {spearman.correlation: .2f}, p-value: {spearman.pvalue}")
print(f"Kendall's Tau: {kendall.correlation: .2f}, p-value: {kendall.pvalue}")

# Tady se mi nepovedlo dobrat výsledku. 
# Dle hodnot z tabulky budou pravděpodobně věk embrya a úspěšnost transferu na sobě statisticky nezávislé.

Spearman:  nan, p-value: nan
Kendall's Tau:  nan, p-value: nan
